In [1]:
import os
import sys
import pandas as pd
import chromadb
from chromadb.config import Settings
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
df = pd.read_csv('./data/job_opening.csv')

In [3]:
# 문서 로드
documents = [Document(page_content=f"공고ID: {row['rec_idx']}, 공고내용: {row['jd_text']}, 공고제목: {row['recruit_title']}, 직무구분: {row['recruit_kewdcdnm']}, 지역: {row['company_place']}, 경력구분: {row['career']}, 학력: {row['education']}", metadata={"id": idx}) for idx, row in df.iterrows()]

In [4]:
# chromadb 연결

client = chromadb.HttpClient(host='43.202.186.183', port=8000, settings=Settings(allow_reset=True, anonymized_telemetry=False))

In [8]:
# openai 임베딩 생성
from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions

load_dotenv()

embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002"
    )


/tmp/ipykernel_606374/1764063953.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [ ]:
#collection 생성

from datetime import datetime

collection = client.create_collection(
    name="job_opening", 
    embedding_function=embeddings,
    metadata={
        "description": "for recommend job_opening",
        "created": str(datetime.now())
    }  
)


In [5]:
collection

NameError: name 'collection' is not defined

In [6]:
documents[0].metadata['id']

0

In [76]:
# 문서 삽입
try:
    for document in documents:
        if document.page_content is not None:
            collection.add(
                documents = [document.page_content],
                ids = [str(document.metadata['id'])]
            )
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


In [9]:
db = Chroma(client=client, collection_name="job_opening",embedding_function=embeddings)

/tmp/ipykernel_606374/3033468460.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(client=client, collection_name="job_opening",embedding_function=embeddings)


In [10]:
retriever = db.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

docs = retriever.invoke("데이터 분석, AI 모델링, 감성 분석, 개발 직무 추천해줘")

In [11]:
docs

[Document(metadata={}, page_content="공고ID: 50139999, 공고내용: \n채용공고 상세\n[코스닥기업] 업무시스템 개발자 채용2005년 04월 02일에 설립된 응용 소프트웨어 개발 및 공급업업종의 AI기반데이터 분석,로우코드솔루션 개발,공급사업을 하는 코스닥,중소기업,주식회사기업 입니다.모집부문 및 상세내용모집부문상세내용업무시스템 개발자2명ㆍ솔루션 기반 업무시스템 개발ㆍ솔루션 개발 리더자격요건주요업무담당업무ㆍ자격요건ㆍ관련 경력 5년 이상ㆍ DB, SQL 및 데이터 지식 보유자ㆍjavascript 사용 가능자우대사항ㆍ동종업계 관련 업무 경험자ㆍ솔루션을 활용한 포털 및 업무시스템 구축 경험자ㆍ고객사 사용자 관점의 우수한 커뮤니케이션 스킬 보유자\nㆍ기타 필수 사항\n우대사항근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:ㆍ근무지역:(135-080) 서울 강남구 역삼동 708-8 세방빌딩 신관17층 -\xa0서울 2호선 선릉역에서 1분 이내전형절차 서류전형(※서류합격자에 한하여간단한 전화인터뷰 진행) 1차면접 2차면접 최종합격접수기간 및 방법ㆍ접수기간:2025년 3월 4일 (화) 17시~ 2025년 3월 31일 (월) 24시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:이력서\xa0및 자기소개서(사진, 연락처, 희망연봉 반드시 기재요망)경력기술서 (프로젝트별 Skill inventory 포함 우대)ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.\n, 공고제목: [코스닥기업] 업무시스템 개발자 채용, 직무구분: ['웹개발', '데이터시각화', '솔루션', '컨설턴트', 'BI 엔지니어'], 지역: 서울 강남구, 경력구분: 경력 5년↑ · 정규직, 학력: 대학(2,3년)↑"),
 Document(metadata={}, page_content="공고ID: 50197577

In [94]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(
        model="gpt-4",
        temperature=0
    )

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(
    """
    당신은 채용 공고 추천 전문가입니다.
    다음은 관련 자료입니다:
    {docs}

    그리고 사용자 정보는 다음과 같습니다:
    - 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발
    - 관심 산업/도메인: IT, 빅데이터
    - 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit
    - 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입
    - 희망 지역: 서울특별시 강남구

    ### 출력 형식 예시
    1. **직무명**: AI 분석가 (신입)
    - **산업 분야**: IT 서비스 / 빅데이터 분석
    - **필요 기술**: Python, TensorFlow, SQL, Pandas
    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999
    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.

    이 정보를 바탕으로 사용자 정보에 가장 적합한 채용 공고 5개를 추천해주세요.
    출력 형식과 동일한 형식으로 출력해주세요.
    
    """

    )

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [101]:
# 체인 생성
chain = LLMChain(llm=llm, prompt=chat_prompt_template)

# invoke 호출
response = chain.invoke({
    "docs": docs
})
result = response['text']

In [103]:
result.split('\n')

['1. **직무명**: [코스닥기업] 업무시스템 개발자 채용',
 '    - **산업 분야**: IT 서비스 / 응용 소프트웨어 개발 및 공급업',
 '    - **필요 기술**: DB, SQL, JavaScript',
 '    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999',
 '    - **추천 사유**: 사용자의 데이터 분석 및 AI 모델링 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.',
 '',
 '2. **직무명**: 경동도시가스 데이터 분석 및 관련 사업개발 전문가 채용',
 '    - **산업 분야**: IT 서비스 / 데이터 분석',
 '    - **필요 기술**: Python, R, JAVA, DB 분석',
 '    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50197577',
 '    - **추천 사유**: 사용자의 데이터 분석 및 AI 모델링 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다.',
 '',
 '3. **직무명**: 음성인식 AI 혁신을 이끌 [서비스기획 팀장] 모집',
 '    - **산업 분야**: IT 서비스 / AI 솔루션',
 '    - **필요 기술**: 서비스 기획 경험',
 '    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50282974',
 '    - **추천 사유**: 사용자의 AI 모델링 역량과 유사한 프로젝트 경험이 있고, 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.',
 '',
 '4. **직무명**: 음성인식 AI 혁신을 이끌 [서비스기획 담당자] 모집',
 '    - **